In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime, timedelta, timezone
import pytz
import scipy.stats
from dateutil.relativedelta import relativedelta

import warnings

# Suppress the specific warnings
warnings.filterwarnings("ignore")

today = datetime.utcnow()

from api_keys import espn_link

In [2]:
today = today - timedelta(days=1)

In [3]:


homeOnlySked = pd.read_csv('data/sked.csv', index_col=False)
homeOnlySked

completeSked = pd.read_csv('data/sked_full.csv', index_col=False)
completeSked

team_names = pd.read_csv('data/team_names.csv', index_col=False)
team_names

file_name = f"data/allG_df_fp-{today.strftime('%Y-%m-%d')}.csv"
allG_df = pd.read_csv(file_name, index_col=False)
allG_df

file_name = f"data/all_df_fp-{today.strftime('%Y-%m-%d')}.csv"
all_df = pd.read_csv(file_name, index_col=False)
all_df

team_names_only = team_names[['abbreviation', 'name']]

all_df_forwards =  all_df.loc[all_df['position'] != 'D']
all_df_defense =  all_df.loc[all_df['position'] == 'D']

all_df_centers =  all_df.loc[all_df['secondaryPosition'] == 'C']
all_df_wingers =  all_df.loc[all_df['secondaryPosition'] == 'W']

file_name = f"data/summary_stats-{today.strftime('%Y-%m-%d')}.csv"
player_bios = pd.read_csv(file_name, index_col=False)

file_name = f"data/rankings_worksheet_{(today-timedelta(days=6)).strftime('%Y-%m-%d')}.xlsx"
rankings = pd.read_excel(file_name)

last_season = pd.read_csv('data/2022-23.csv')

In [4]:
single_games = all_df.sort_values('fantasyPoints', ascending=False)

goaltenders_single = single_games.loc[single_games['position'] == 'G']
skaters_single = single_games.loc[single_games['position'] != 'G']
defense_single = skaters_single.loc[skaters_single['position'] == 'D']
forwards_single = skaters_single.loc[skaters_single['position'] != 'D']

In [5]:
defense_single.columns.to_list()

['playerId',
 'sweaterNumber',
 'name',
 'position',
 'goals',
 'assists',
 'points',
 'plusMinus',
 'pim',
 'hits',
 'blockedShots',
 'powerPlayGoals',
 'powerPlayPoints',
 'shorthandedGoals',
 'shPoints',
 'shots',
 'faceoffs',
 'toi',
 'powerPlayToi',
 'shorthandedToi',
 'team',
 'opponent',
 'secondaryPosition',
 'tertiaryPosition',
 'gameDate',
 'gameTime',
 'gameId',
 'gamesPlayed',
 'fantasyPoints',
 'specialTeams']

In [6]:
import pymannkendall as mk

def convert_to_minutes_seconds(seconds):
    minutes = seconds // 60
    remaining_seconds = seconds % 60
    return f"{minutes}:{remaining_seconds:02d}"


test_data = defense_single.loc[defense_single['name'] == 'Tony DeAngelo'].sort_values('gameDate', ascending=True)
test_data = test_data['toi'].to_list()
result = mk.original_test(test_data)
print(f"{d}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
print(', '.join(formatted_times))

ZeroDivisionError: float division by zero

In [7]:
d_cull = defense_single['name'].value_counts()
d_cull = d_cull[d_cull == 1]
d_cull = d_cull.index.to_list()
d_list = defense_single['name'].unique()
d_list = d_list.tolist()
d_list = [name for name in d_list if name not in d_cull]

In [8]:

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['toi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))

print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['fantasyPoints'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s fantasy points are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        
print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['powerPlayToi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        
print('-'*20)

for d in d_list:
    test_data = defense_single.loc[defense_single['name'] == d].sort_values('gameDate', ascending=True)
    test_data = test_data['shots'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{d}'s shots are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num}' for num in test_data]
        print(', '.join(formatted_numbers))

J. Trouba's ice time is increasing and the Tau score is 0.44
21:13, 20:32, 20:43, 17:32, 21:27, 22:33, 19:29, 20:50, 23:54, 24:56, 26:26, 24:50, 25:12, 22:54, 21:40, 23:57, 23:52, 27:36, 21:54
K. Guhle's ice time is increasing and the Tau score is 0.38
19:24, 17:03, 10:19, 25:08, 26:08, 20:25, 18:33, 19:53, 21:16, 21:12, 20:20, 20:06, 21:37, 24:27, 22:53, 25:01, 22:14
N. Mikkola's ice time is increasing and the Tau score is 0.35
18:03, 15:13, 19:43, 19:47, 19:53, 20:26, 22:13, 19:37, 17:30, 19:09, 20:52, 21:53, 22:45, 22:37, 24:36, 18:44, 23:17, 20:11, 18:21, 22:27
O. Ekman-Larsson's ice time is decreasing and the Tau score is -0.52
26:32, 26:16, 25:19, 23:56, 22:04, 22:37, 23:53, 20:22, 26:24, 23:56, 22:21, 23:53, 21:10, 20:37, 23:48, 28:22, 17:59, 18:50, 16:13, 17:47
E. Gustafsson's ice time is increasing and the Tau score is 0.43
15:33, 21:56, 15:42, 18:58, 17:23, 17:15, 18:14, 16:15, 18:11, 18:42, 20:02, 18:43, 20:25, 26:18, 20:24, 19:53, 21:02, 17:59, 20:39
T. Myers's ice time is 

--------------------
V. Hedman's shots are decreasing and the Tau score is -0.37
4, 3, 2, 3, 1, 1, 3, 1, 2, 2, 1, 2, 1, 1, 2, 0, 1, 1, 5, 1, 0
O. Ekman-Larsson's shots are decreasing and the Tau score is -0.42
4, 3, 1, 3, 3, 1, 2, 4, 0, 2, 3, 1, 0, 2, 0, 1, 1, 1, 0, 1
T. Krug's shots are increasing and the Tau score is 0.33
2, 1, 1, 1, 2, 1, 1, 3, 0, 1, 1, 6, 2, 1, 6, 4, 2, 3, 4
D. Samberg's shots are increasing and the Tau score is 0.39
0, 0, 1, 0, 0, 1, 1, 2, 2, 0, 0, 2, 0, 1, 2, 1, 1, 2, 3
C. Murphy's shots are increasing and the Tau score is 0.33
0, 0, 0, 0, 0, 1, 1, 0, 2, 1, 2, 3, 1, 0, 0, 4, 4, 0
C. Tanev's shots are decreasing and the Tau score is -0.34
1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 0, 0, 2, 0, 0, 0, 1, 2, 0, 1, 0
C. Clifton's shots are decreasing and the Tau score is -0.4
2, 2, 1, 1, 2, 1, 3, 0, 0, 1, 1, 0, 0, 3, 1, 0, 0, 0, 0
S. Benoit's shots are decreasing and the Tau score is -0.73
2, 1, 1, 1, 0, 0


In [9]:
f_cull = forwards_single['name'].value_counts()
f_cull = f_cull[f_cull == 1]
f_cull = f_cull.index.to_list()
f_list = forwards_single['name'].unique()
f_list = f_list.tolist()
f_list = [name for name in f_list if name not in f_cull]

In [10]:
#Print a list of forward trends
    
for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['toi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))

print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['fantasyPoints'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s fantasy points are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num:.1f}' for num in test_data]
        print(', '.join(formatted_numbers))
        
print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['powerPlayToi'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s power-play ice time is {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_times = [convert_to_minutes_seconds(seconds) for seconds in test_data]
        print(', '.join(formatted_times))
        
print('-'*20)

for f in f_list:
    test_data = forwards_single.loc[forwards_single['name'] == f].sort_values('gameDate', ascending=True)
    test_data = test_data['shots'].to_list()
    result = mk.original_test(test_data)
    if result.trend != 'no trend':
        print(f"{f}'s shots are {result.trend} and the Tau score is {round(result.Tau, 2)}")
        formatted_numbers = [f'{num}' for num in test_data]
        print(', '.join(formatted_numbers))

K. Connor's ice time is decreasing and the Tau score is -0.34
22:00, 21:50, 21:37, 21:37, 20:56, 18:56, 22:07, 23:18, 23:16, 21:17, 17:53, 16:46, 18:28, 20:55, 22:45, 20:07, 18:09, 20:34, 18:29
N. Niederreiter's ice time is increasing and the Tau score is 0.38
15:16, 10:57, 13:20, 14:37, 17:48, 14:53, 15:18, 14:11, 14:56, 15:41, 16:05, 14:58, 14:57, 11:30, 17:13, 16:50, 16:35, 16:55, 15:59
E. Rodrigues's ice time is decreasing and the Tau score is -0.42
19:53, 19:12, 15:53, 20:38, 16:54, 19:45, 18:23, 21:08, 16:25, 14:43, 17:39, 18:01, 16:18, 14:04, 18:46, 17:19, 17:02, 12:24, 15:58, 14:16
C. Keller's ice time is decreasing and the Tau score is -0.35
20:55, 22:03, 21:40, 15:47, 21:20, 22:46, 19:05, 16:52, 21:09, 19:20, 20:10, 20:20, 18:30, 18:07, 19:13, 19:25, 21:14, 18:44, 17:28, 16:34
D. Pastrnak's ice time is increasing and the Tau score is 0.38
17:24, 17:26, 18:09, 18:18, 18:46, 17:29, 20:10, 20:25, 21:17, 24:25, 18:56, 18:53, 20:07, 22:06, 17:35, 13:13, 20:31, 19:13, 21:24, 21:12


Z. Girgensons's ice time is increasing and the Tau score is 0.36
8:48, 10:07, 9:11, 11:34, 13:05, 12:09, 16:21, 11:11, 12:03, 14:01, 14:17, 13:05, 13:49, 14:29, 14:30, 15:48, 12:34, 13:23, 14:07, 6:35
W. Cuylle's ice time is decreasing and the Tau score is -0.47
12:34, 14:54, 11:56, 13:15, 14:06, 13:42, 12:53, 12:16, 11:41, 11:49, 9:59, 12:13, 14:21, 13:00, 12:11, 10:03, 8:45, 6:01, 11:33
N. Cousins's ice time is increasing and the Tau score is 0.39
10:31, 7:06, 11:07, 10:08, 7:53, 14:35, 15:19, 13:00, 11:51, 13:46, 12:50, 14:41, 13:36, 12:59, 13:26, 12:32, 13:54, 13:26, 14:15, 14:33
J. Martinook's ice time is decreasing and the Tau score is -0.35
15:41, 16:52, 14:53, 15:56, 18:21, 15:03, 13:58, 15:15, 13:32, 15:43, 12:38, 16:05, 11:23, 15:52, 15:41, 13:57, 13:21, 14:34, 13:14
M. Eyssimont's ice time is increasing and the Tau score is 0.42
8:31, 11:39, 13:57, 10:34, 10:59, 11:08, 9:09, 10:39, 10:33, 7:25, 10:01, 11:25, 10:14, 12:04, 12:47, 12:43, 12:57, 14:55, 14:44, 14:31
L. Reichel's

N. Schmaltz's power-play ice time is decreasing and the Tau score is -0.38
7:12, 5:27, 3:27, 1:47, 5:37, 5:16, 3:06, 3:48, 5:08, 3:41, 3:30, 3:49, 2:24, 4:30, 2:12, 1:15, 3:17, 6:22, 1:25, 1:43
M. Hoffman's power-play ice time is decreasing and the Tau score is -0.43
2:04, 5:01, 6:29, 2:46, 3:11, 3:07, 0:39, 2:00, 0:00, 2:10, 0:52, 0:00, 2:42, 0:28, 1:22, 0:59, 1:22, 1:13, 0:21, 0:48
J. Compher's power-play ice time is increasing and the Tau score is 0.44
1:14, 0:55, 1:41, 0:00, 0:50, 2:37, 3:08, 1:42, 2:56, 1:04, 1:23, 2:06, 4:33, 2:09, 2:14, 1:02, 3:57, 2:58, 4:56
A. Kempe's power-play ice time is decreasing and the Tau score is -0.47
4:00, 5:32, 4:44, 0:00, 5:34, 5:07, 6:22, 4:28, 3:46, 5:05, 3:24, 3:23, 2:54, 3:33, 1:58, 1:48, 3:45, 2:51, 2:01
A. Duclair's power-play ice time is decreasing and the Tau score is -0.37
1:48, 4:38, 6:51, 2:13, 3:03, 3:09, 0:57, 2:00, 0:00, 3:32, 6:29, 3:38, 2:38, 0:56, 1:12, 1:09, 0:00, 0:00
M. Joseph's power-play ice time is increasing and the Tau sco

C. Brown's power-play ice time is decreasing and the Tau score is -0.42
1:04, 0:41, 1:29, 1:33, 1:19, 1:35, 0:57, 0:48, 0:26, 0:00, 1:26, 0:00, 0:00
--------------------
T. Terry's shots are decreasing and the Tau score is -0.41
2, 2, 4, 6, 2, 3, 2, 1, 2, 6, 5, 2, 2, 3, 0, 1, 1, 0, 0, 1
S. Reinhart's shots are decreasing and the Tau score is -0.53
2, 4, 5, 4, 4, 4, 3, 5, 2, 2, 2, 2, 4, 3, 3, 1, 1, 0, 0, 1
T. Thompson's shots are decreasing and the Tau score is -0.41
3, 4, 5, 2, 10, 5, 3, 2, 2, 1, 3, 7, 2, 2, 0, 1
E. Malkin's shots are decreasing and the Tau score is -0.32
6, 4, 5, 3, 2, 3, 2, 0, 5, 2, 4, 0, 2, 2, 0, 2, 2, 2, 2, 3
K. Hayes's shots are increasing and the Tau score is 0.44
0, 1, 0, 1, 1, 1, 1, 1, 3, 1, 1, 2, 0, 2, 3, 5, 1, 3, 2
A. Lafrenière's shots are increasing and the Tau score is 0.38
3, 1, 0, 0, 1, 2, 3, 1, 1, 2, 2, 2, 4, 2, 3, 3, 6, 0, 4
L. Crouse's shots are increasing and the Tau score is 0.36
1, 1, 0, 2, 0, 4, 1, 3, 4, 3, 1, 2, 0, 3, 5, 2, 7, 4, 3
A. Copp's shot

In [19]:
hurricanes = all_df.loc[all_df['team'] == 'CAR'].sort_values('gameDate')
hurricanes_pp = hurricanes.groupby('gameDate').sum()

formatted_times = [convert_to_minutes_seconds(seconds) for seconds in hurricanes_pp['powerPlayToi']]

print(', '.join(formatted_times))


55:15, 40:00, 51:35, 47:10, 13:42, 47:45, 28:26, 10:00, 33:30, 4:45, 11:45, 24:00, 32:00, 24:00, 21:35, 30:00, 22:55


In [22]:
def summary_statistics(df):
    temp_build = df.groupby('playerId').agg({
        'name': 'first',
        'team': 'last',
        'position': 'first',
        'toi': 'sum',
        'gamesPlayed': 'sum',
        'goals': 'sum',
        'assists': 'sum',
        'specialTeams': 'sum',
        'shots': 'sum',
        'hits': 'sum',
        'blockedShots': 'sum',
        'powerPlayToi': 'sum',
        'shorthandedToi': 'sum',
        'plusMinus': 'sum',
        'fantasyPoints': 'sum',
        'secondaryPosition': 'first',
        'tertiaryPosition': 'first'
    }).reset_index()



    temp_build['FPP60'] = (temp_build['fantasyPoints'] / temp_build['toi'] * 3600).round(2)
    temp_build['FPPG'] = (temp_build['fantasyPoints'] / temp_build['gamesPlayed']).round(2)

    temp_build = temp_build.sort_values(by='FPPG', ascending=False)
    
    return temp_build

In [23]:
hurricanes = all_df.loc[all_df['team'] == 'CAR'].sort_values('gameDate')
summary_statistics(hurricanes)

,playerId,name,team,position,toi,gamesPlayed,goals,assists,specialTeams,shots,hits,blockedShots,powerPlayToi,shorthandedToi,plusMinus,fantasyPoints,secondaryPosition,tertiaryPosition,FPP60,FPPG
13,8478427,S. Aho,CAR,C,17034,14,5,10,7,43,2,4,2733,1601,-1,30.0,C,F,6.34,2.14
8,8476958,J. Slavin,CAR,D,21547,17,3,7,2,46,7,29,93,3137,8,33.8,D,D,5.65,1.99
19,8482093,S. Jarvis,CAR,C,19780,17,7,6,7,40,24,6,3604,1791,-3,32.9,C,F,5.99,1.94
5,8476869,B. Skjei,CAR,D,21467,17,2,10,2,34,9,23,208,2378,6,30.8,D,D,5.17,1.81
9,8477488,B. Pesce,CAR,D,10792,9,1,1,0,12,6,23,130,1348,3,16.3,D,D,5.44,1.81
6,8476882,T. Teravainen,CAR,L,16717,17,9,3,6,37,4,5,3117,1823,-2,30.6,W,F,6.59,1.80
16,8480829,J. Kotkaniemi,CAR,C,15590,17,6,7,3,38,14,7,2250,125,-2,29.2,C,F,6.74,1.72
15,8480039,M. Necas,CAR,C,18517,17,5,7,4,41,16,6,3485,84,-9,27.7,C,F,5.39,1.63
0,8470613,B. Burns,CAR,D,22286,17,4,4,3,41,4,18,2445,2530,5,27.0,D,D,4.36,1.59
12,8478047,M. Bunting,CAR,L,15252,16,3,6,4,25,5,5,3084,16,-4,19.5,W,F,4.60,1.22
